# Using Row Access Policies with Cortex Agents
This Notebook will help set up a Row Access Policy (RAP) on the underlying data that is accessed via the Cortex Agents API to illustrate how we can restrict a user or tenant's access to data in Snowflake in a way that allows us to use the SQL generated by Cortex Agent.

This Notebook uses the setup in the [Getting Started with Cortex Agents](https://quickstarts.snowflake.com/guide/getting_started_with_cortex_agents/index.html) Quickstart. We'll focus on protecting the `SALES_METRICS` table based on the `SALES_REP` column.

The primary challenge is that the SQL that is generated by Cortex Analyst (within the context of Cortex Agent) is returned as a string. We don't want to depend on the LLM adding the necessary predicates to restrict the user or tenant to just their data, so we either need to modify the SQL string or find another way. Modifying the SQL would require parsing the SQL and it can get complex. The option we consider is using Row Access Policies to impose that restriction to tenant data. 

For this multi-user/multi-tenant scenario, we will use a session variable (we'll use `TENANT`) and use the value of that session variable in the Row Access Policy. We will protect all applicable tables with a Row Access Policy based on this sesison variable. The approach is that we will set the value of the `TENANT` session variable to be the user or tenant and then execute the generated SQL.

The application will connect to Snowflake using a service user (with a service role), issue Cortex Agents API calls, and when issuing SQL will first set the session variable and then submit the SQL. The example code for this applicaiton can be found [here](http://TODO-GET-ACTUAL-URL).

A note about the Cortex Agent API. The legacy behavior of running the Cortex Agent API was that whenever SQL is needed to be run, Cortex Agent will return the results so far, ending with a `tool_use` message that includes SQL. The client needs to look at the response to see if SQL was returned, and if so it needs to run that SQL, note the Query ID of the SQL statement, and re-issue the Cortex Agent API call appending a new message that includes the Query ID of the SQL. At this point, Cortex Agent will continue its analysis and return the next response. In this legacy behavior, the client needs to supply the full agent YAML spec every time, and does not use named Agents.

## 1. Create the necessary roles
First we create the application role.

In [ ]:
USE ROLE securityadmin;
CREATE ROLE IF NOT EXISTS rl_app;
GRANT ROLE sales_intelligence_rl TO ROLE rl_app;

SET cur_user = CURRENT_USER();
GRANT ROLE rl_app TO USER IDENTIFIER($cur_user);

## 2. Create the Entitlements Table
Next, we will create the entitlements table that will map which application users will have access to which sales rep's data.

In [ ]:
USE ROLE securityadmin;
GRANT ALL ON SCHEMA sales_intelligence.data TO ROLE sales_intelligence_rl; -- needed to create entitlements table and RAP

USE ROLE sales_intelligence_rl;
USE SCHEMA sales_intelligence.data;

-- Each rep will have access to their own data (user ID is their first name).
-- Alice has access to Sarah and Rachel's data. Bob has access to James and Mike's data.
CREATE OR REPLACE TABLE sales_entitlement(userid VARCHAR, sales_rep VARCHAR);
INSERT INTO sales_entitlement(userid, sales_rep) VALUES
    ('James', 'James Wilson'),
    ('Mike', 'Mike Chen'),
    ('Rachel', 'Rachel Torres'),
    ('Sarah', 'Sarah Johnson'),
    ('Bob', 'James Wilson'),
    ('Bob', 'Mike Chen'),
    ('Alice', 'Rachel Torres'),
    ('Alice', 'Sarah Johnson')
;

## 3. Set up the Row Access Policy
In order to make the Row Access Policy performant, we will create a memoizable function that collects all `SALES_REP`s that a given `USERID` has access to. Then, we create a Row Access Policy that uses that function, passing in the value of the session variable (retrieved via the `GETVARIABLE()` function) as the argument. Since this is constant for the length of the query, the UDF will run once and the result reused (as is the behavior of memoizable functions).

In [ ]:
USE ROLE sales_intelligence_rl;
USE SCHEMA sales_intelligence.data;

DROP FUNCTION IF EXISTS sales_reps(STRING);
CREATE FUNCTION sales_reps(who STRING)
    RETURNS ARRAY
    MEMOIZABLE
    AS 'SELECT ARRAY_AGG(sales_rep) FROM sales_entitlement WHERE userid = who'
;

DROP ROW ACCESS POLICY IF EXISTS rap_sales_rep;
CREATE ROW ACCESS POLICY rap_sales_rep
    AS (rep VARCHAR)
    RETURNS BOOLEAN ->
        CASE 
            WHEN 'RL_APP' = CURRENT_ROLE() THEN  ARRAY_CONTAINS(rep::VARIANT, sales_reps(GETVARIABLE('TENANT')))
            ELSE TRUE
        END        
;

Now we add this Row Access Policy to the `SALES_REP` column of the `SALES_METRICS` table.

In [ ]:
USE ROLE sales_intelligence_rl;
USE SCHEMA sales_intelligence.data;

ALTER TABLE sales_metrics DROP ALL ROW ACCESS POLICIES; -- Just to make sure there aren't any others
ALTER TABLE sales_metrics ADD ROW ACCESS POLICY rap_sales_rep ON (sales_rep);

## 4. Testing
Now we can test the access to the `SALES_METRICS` table. First, let's use the `SALES_INTELLIGENCE_RL` and ensure all data is accessible;

In [ ]:
USE ROLE sales_intelligence_rl;
USE SCHEMA sales_intelligence.data;

SELECT sales_rep, COUNT(*) AS ct FROM sales_metrics GROUP BY sales_rep ORDER BY sales_rep;

Next, let's use the `RL_APP` role and set `TENANT` variable to `Sarah` to see that only Sarah's data is accessible.

In [ ]:
USE ROLE rl_app;
USE SCHEMA sales_intelligence.data;

SET TENANT = 'Sarah';

SELECT sales_rep, COUNT(*) AS ct FROM sales_metrics GROUP BY sales_rep ORDER BY sales_rep;

Feel free to change the value of `TENANT` to `James`, `Mike`, `Rachel`, `Alice` or `Bob` to see the different access.

## 5. Application
At this point we can turn our attention to an application that will connect to Snowflake and use the session variable to set the tenant and restrict the access to the various tenants. 

See the next steps in the Github repo for a sample application.

One thing we will need is a Programmatic Access Token (PAT) that allows the application to connect to Snowflake using the `RL_APP` role. We can generate one as follows. Note that you must meet the necessary requirements for creating a PAT token (see [here](https://docs.snowflake.com/user-guide/programmatic-access-tokens)).

In [ ]:
USE ROLE securityadmin;
ALTER USER ADD PROGRAMMATIC ACCESS TOKEN demo_agent ROLE_RESTRICTION = 'RL_APP';

You will need to set a number of environment variables in order to use the example application. Run the following cell to generate the command that you should copy-paste into your terminal before running the application.

In [ ]:
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.connector import DictCursor
pat = sql_pat.to_pandas().iloc[0].to_dict()['token_secret']
session = get_active_session()
cur = session.connection.cursor(DictCursor)
d = cur.execute("SELECT current_account() AS account").fetchall()
account = d[0]["ACCOUNT"]

st.markdown(f"""
```
export NEXT_PUBLIC_SNOWFLAKE_URL="https://{account}.snowflakecomputing.com"
export NEXT_PUBLIC_SEMANTIC_MODEL_PATH="@SALES_INTELLIGENCE.DATA.MODELS/sales_metrics_model.yaml"
export NEXT_PUBLIC_SEARCH_SERVICE_PATH="SALES_INTELLIGENCE.DATA.SALES_CONVERSATION_SEARCH"
export NEXT_PUBLIC_SNOWFLAKE_PAT="{pat}"
```
""")

In the application, you will first be presented with a login screen. The users are available by pulldown, and the passwords are the same as the username (i.e., the password for `Alice` is `Alice`).

Once you log in, you can ask a question, and the results will be restricted to the data that the user has access to. An example question to ask is

* `What are the biggest deals won and lost?`

Try logging out and logging in as a different user and asking the same question to see how the answer changes.